#### Fluent API for Time Series with TimeDB

This notebook demonstrates the new fluent collection-based API:
1. Creating a TimeDataClient
2. Using series collections for targeted and broad operations
3. Filtering with .where() to narrow down selections
4. Reading and writing without managing series IDs directly

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from timedb import TimeDataClient
load_dotenv()

td = TimeDataClient()

## Part 1: Setup - Create Schema and Series

First, let's create a fresh database with multiple series that have different labels.

In [2]:
td.delete()
td.create()

Creating database schema...
✓ Schema created successfully


In [3]:
# Create multiple series with labels
series_configs = [
    # Wind turbines at different sites
    {'name': 'wind_power', 'unit': 'MW', 'labels': {'turbine': 'T01', 'site': 'Gotland', 'type': 'onshore'}},
    {'name': 'wind_power', 'unit': 'MW', 'labels': {'turbine': 'T02', 'site': 'Gotland', 'type': 'onshore'}},
    {'name': 'wind_power', 'unit': 'MW', 'labels': {'turbine': 'T03', 'site': 'Gotland', 'type': 'offshore'}},
    {'name': 'wind_power', 'unit': 'MW', 'labels': {'turbine': 'T01', 'site': 'Skåne', 'type': 'onshore'}},
    
    # Solar installations
    {'name': 'solar_power', 'unit': 'MW', 'labels': {'panel_id': 'P01', 'site': 'Gotland'}},
    {'name': 'solar_power', 'unit': 'MW', 'labels': {'panel_id': 'P02', 'site': 'Skåne'}},
]

for config in series_configs:
    td.create_series(**config)

## Part 2: Targeted Action - Write to a Specific Turbine

Use the fluent API to write data to a specific turbine without managing series IDs.

In [4]:
# Prepare data for a single turbine
base_time = datetime(2025, 1, 1, 0, 0, tzinfo=timezone.utc)
dates = [base_time + timedelta(hours=i) for i in range(24)]

df_t01 = pd.DataFrame({
    'valid_time': dates,
    'power': [50.0 + i * 1.5 for i in range(24)]  # Gradual increase
})

# Write to a specific turbine using fluent API
result = td.series("wind_power").where(site="Gotland", turbine="T01").insert_batch(df_t01)

## Part 3: Broad Action - Multiple Series at Once

Get a collection that matches multiple series and explore it.

In [5]:
# Get a collection that matches all onshore turbines at Gotland
gotland_onshore = td.series("wind_power").where(site="Gotland", type="onshore")

# Check how many series match
print(f"Collection matches {gotland_onshore.count()} series")
print(f"Turbines: {gotland_onshore.list_labels('turbine')}")

Collection matches 2 series
Turbines: ['T01', 'T02']


## Part 4: Progressive Filtering

Start broad and narrow down step by step.

In [6]:
# Start broad and narrow down step by step
wind = td.series("wind_power")
print(f"All wind power series: {wind.count()}")

gotland_wind = wind.where(site="Gotland")
print(f"Wind at Gotland: {gotland_wind.count()}")

gotland_onshore = gotland_wind.where(type="onshore")
print(f"Onshore wind at Gotland: {gotland_onshore.count()}")
print(f"Turbines: {gotland_onshore.list_labels('turbine')}")

All wind power series: 4
Wind at Gotland: 3
Onshore wind at Gotland: 2
Turbines: ['T01', 'T02', 'T03']


## Part 5: Reading Data

Read data back using the fluent API.

In [7]:
# Read data for a specific turbine
df_t01 = td.series("wind_power").where(site="Gotland", turbine="T01").read()
print("Data for T01 at Gotland:")
print(df_t01.head())

# Read data for all turbines at Gotland
df_gotland = td.series("wind_power").where(site="Gotland").read()
print("\nData for all turbines at Gotland:")
print(df_gotland.head())

Data for T01 at Gotland:
name                       wind_power
valid_time                           
2025-01-01 00:00:00+00:00        50.0
2025-01-01 01:00:00+00:00        51.5
2025-01-01 02:00:00+00:00        53.0
2025-01-01 03:00:00+00:00        54.5
2025-01-01 04:00:00+00:00        56.0

Data for all turbines at Gotland:
name                       wind_power
valid_time                           
2025-01-01 00:00:00+00:00        50.0
2025-01-01 01:00:00+00:00        51.5
2025-01-01 02:00:00+00:00        53.0
2025-01-01 03:00:00+00:00        54.5
2025-01-01 04:00:00+00:00        56.0


## Part 6: Exploration

Discover what series exist using the fluent API.

In [8]:
# Discover what exists
wind = td.series("wind_power")

sites = wind.list_labels("site")
print(f"Sites with wind power: {sites}")

for site in sites:
    site_wind = wind.where(site=site)
    count = site_wind.count()
    turbines = site_wind.list_labels("turbine")
    print(f"\n{site}: {count} turbines - {turbines}")

Sites with wind power: ['Gotland', 'Skåne']

Gotland: 3 turbines - ['T01', 'T02', 'T03']

Skåne: 1 turbines - ['T01', 'T02', 'T03']


## Part 7: Error Handling and Edge Cases

The fluent API provides clear error messages when used incorrectly. Let's explore common mistakes and how the API handles them.

### 7.1: Wrong DataFrame Format - Multiple Columns for Single Series

If you provide a DataFrame with multiple data columns but the collection resolves to a single series, you'll get a clear error.

In [9]:
# Create a DataFrame with TWO data columns
df_wrong = pd.DataFrame({
    'valid_time': dates,
    'power_1': [10.0] * 24,
    'power_2': [20.0] * 24,
})

# Try to insert into a single series (specific turbine)
try:
    td.series("wind_power").where(site="Gotland", turbine="T01").insert_batch(df_wrong)
except ValueError as e:
    print(f"❌ Error caught: {e}")
    print("\n✓ The API correctly detected the mismatch!")

❌ Error caught: DataFrame has 2 series columns, but collection resolves to 1 series. Expected exactly 1 data column (excluding time columns).

✓ The API correctly detected the mismatch!


### 7.2: No Series Found - Filters Don't Match Anything

If your filters don't match any existing series, you'll get a helpful error message.

In [10]:
# Try to read from a non-existent site
try:
    df = td.series("wind_power").where(site="NonExistentSite").read()
except ValueError as e:
    print(f"❌ Error caught: {e}")
    print("\n✓ The API tells you exactly what filters didn't match!")

# Try to write to a non-existent series
df_single = pd.DataFrame({
    'valid_time': dates,
    'power': [100.0] * 24,
})

try:
    td.series("wind_power").where(site="Mars", turbine="T99").insert_batch(df_single)
except ValueError as e:
    print(f"\n❌ Error caught: {e}")
    print("\n✓ Clear feedback about what went wrong!")

❌ Error caught: No series found matching filters: name=wind_power, unit=None, labels={'site': 'NonExistentSite'}

✓ The API tells you exactly what filters didn't match!

❌ Error caught: No series found matching filters: name=wind_power, unit=None, labels={'site': 'Mars', 'turbine': 'T99'}

✓ Clear feedback about what went wrong!


### 7.3: Multi-Series Collections - Understanding the Behavior

When a collection matches multiple series, the behavior depends on your DataFrame structure.

In [11]:
# When a collection matches MULTIPLE series
broad_collection = td.series("wind_power").where(site="Gotland")
print(f"This collection matches {broad_collection.count()} series")

# Case 1: Single column DataFrame - will use auto-detection
# The column name becomes the series name
df_single_col = pd.DataFrame({
    'valid_time': dates,
    'power': [75.0] * 24,
})

result = broad_collection.insert_batch(df_single_col)
print(f"\n✓ Single-column insert succeeded")
print(f"  Series used: {list(result.series_ids.keys())}")
print(f"  Note: Column 'power' was auto-detected as series name")

# Case 2: Multi-column DataFrame - maps columns to series names
# For proper multi-series write, name columns after turbines
df_multi_col = pd.DataFrame({
    'valid_time': dates,
    'T01': [80.0] * 24,
    'T02': [85.0] * 24,
    'T03': [82.0] * 24,
})

result2 = broad_collection.insert_batch(df_multi_col)
print(f"\n✓ Multi-column insert succeeded")
print(f"  Series used: {list(result2.series_ids.keys())}")
print(f"  Note: Each column matched to a series by name")

This collection matches 3 series

✓ Single-column insert succeeded
  Series used: ['power']
  Note: Column 'power' was auto-detected as series name

✓ Multi-column insert succeeded
  Series used: ['T01', 'T02', 'T03']
  Note: Each column matched to a series by name


### 7.3b: When Column Names Don't Match Series Names

If your DataFrame columns don't match existing series names and you're working with multiple series, you need to be explicit.

In [12]:
# Best practice: Use .count() to ensure you're targeting what you expect
specific = td.series("wind_power").where(site="Gotland", turbine="T01")
print(f"Specific collection: {specific.count()} series")

if specific.count() == 1:
    # Safe to use any column name when there's only one series
    df = pd.DataFrame({
        'valid_time': dates[:5],
        'my_power_reading': [100.0, 101.0, 102.0, 103.0, 104.0]
    })
    result = specific.insert_batch(df)
    print(f"✓ Single-series insert: column name doesn't matter")
    print(f"  Series used: {result.series_ids}")
else:
    print(f"⚠️  Collection matches {specific.count()} series - be explicit!")

Specific collection: 1 series
✓ Single-series insert: column name doesn't matter
  Series used: {'my_power_reading': UUID('574e5e2c-5290-471e-a335-8b974210fe32')}


### 7.4: Missing Time Column

If your DataFrame doesn't have the required time column, you'll get an error.

In [13]:
# DataFrame without the time column
df_no_time = pd.DataFrame({
    'power': [50.0, 60.0, 70.0],
})

try:
    td.series("wind_power").where(site="Gotland", turbine="T01").insert_batch(df_no_time)
except Exception as e:
    print(f"❌ Error caught: {type(e).__name__}: {e}")
    print("\n✓ You need a 'valid_time' column (or specify a different column name)!")

❌ Error caught: ValueError: Column 'valid_time' not found in DataFrame

✓ You need a 'valid_time' column (or specify a different column name)!


### 7.5: Best Practices Summary

To avoid errors:
1. **Single series writes**: Ensure your DataFrame has exactly ONE data column (plus time columns)
2. **Multiple series writes**: DataFrame columns should match series identifiers, or narrow your filter to one series
3. **Check your filters**: Use `.count()` and `.list_labels()` to verify what you're selecting
4. **Always include time**: DataFrames must have a `valid_time` column (or specify `valid_time_col` parameter)
5. **Test incrementally**: Start broad with `.count()`, then narrow down with `.where()`

In [14]:
# Example of good practice: Check before you act
collection = td.series("wind_power").where(site="Gotland")

print(f"About to work with {collection.count()} series:")
turbines = collection.list_labels('turbine')
print(f"Turbines: {turbines}")

# Now narrow it down if needed
specific = collection.where(turbine="T01")
print(f"\nNarrowed down to {specific.count()} series")
print("✓ Ready to insert or read data safely!")

About to work with 3 series:
Turbines: ['T01', 'T02', 'T03']

Narrowed down to 1 series
✓ Ready to insert or read data safely!
